In [1]:
!nvidia-smi

Sun Jul  2 07:10:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.141.03   Driver Version: 470.141.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:0B:00.0 Off |                    0 |
| N/A   29C    P0    43W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# sync python module
%load_ext autoreload
%autoreload 2

import os
import sys

from torchsummaryX import summary

import torch

# set package path
sys.path.append(r'/nfs/Workspace/CardiacSeg')

import argparse
from networks.network import network
from networks.networkx.blocks.cbam import CBAM
from networks.networkx.blocks.ham import HAM
from networks.ssl_head import SSLHead

/opt/conda/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
parser = argparse.ArgumentParser(description="model segmentation pipeline")
args = parser.parse_args([])

args.device = 'cuda'
args.in_channels = 1
args.out_channels = 2
args.roi_x, args.roi_y, args.roi_z = [128, 128, 128]
args.patch_size = 4
args.drop_rate = 0.1
args.kernel_size = 7
args.exp_rate = 4
args.norm_name = 'layer'
args.depths = [3, 3, 9, 3]
args.feature_size = 48         
args.use_init_weights = False
args.deep_sup=True
args.is_conv_stem=False
args.first_feature_size_half=False
args.skip_encoder_name=None #'convmod' #'eca'



net = None
del net
net = network('unetcnx_a1', args) # swinunetr
net.eval()

inp = None
del inp
inp = torch.rand((1, 1, args.roi_x, args.roi_y, args.roi_z)).to(args.device)
summary(net, inp)

# log_dir = '/nfs/Workspace/CardiacSeg/exps/exps/cotr/chgh/tune_results/t_4/main_0ef8c_00000_0_exp=exp_t_4_2023-04-02_10-06-38'
# model_pth = os.path.join(log_dir, 'models', 'final_model.pth')
# ckp = torch.load(model_pth)
# print('final early stop count:', ckp['early_stop_count'])
# print('final epoch:', ckp['epoch'])
# print('best val dice:', ckp['best_acc'])
# net.load_state_dict(ckp["state_dict"])

model: unetcnx_a1
patch size: 4
ker size: 7
exp rate: 4
feature sizes: [48, 96, 192, 384]
depths: [3, 3, 9, 3]
drop rate: 0.1
use init weights: False
is conv stem: False
use deep sup
                                                      Kernel Shape  \
Layer                                                                
0_encoder0.layer.conv1.Conv3d_conv                [1, 48, 3, 3, 3]   
1_encoder0.layer.InstanceNorm3d_norm1                            -   
2_encoder0.layer.LeakyReLU_lrelu                                 -   
3_encoder0.layer.conv2.Conv3d_conv               [48, 48, 3, 3, 3]   
4_encoder0.layer.InstanceNorm3d_norm2                            -   
5_encoder0.layer.conv3.Conv3d_conv                [1, 48, 1, 1, 1]   
6_encoder0.layer.InstanceNorm3d_norm3                            -   
7_encoder0.layer.LeakyReLU_lrelu                                 -   
8_backbone.downsample_layers.0.Conv3d_0           [1, 48, 4, 4, 4]   
9_backbone.downsample_layers.0.LayerNorm_1     

The default value of numeric_only in DataFrame.sum is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.


,Kernel Shape,Output Shape,Params,Mult-Adds
Layer,,,,
0_encoder0.layer.conv1.Conv3d_conv,"[1, 48, 3, 3, 3]","[1, 48, 128, 128, 128]",1296.0,2.717909e+09
1_encoder0.layer.InstanceNorm3d_norm1,-,"[1, 48, 128, 128, 128]",NaN,NaN
2_encoder0.layer.LeakyReLU_lrelu,-,"[1, 48, 128, 128, 128]",NaN,NaN
3_encoder0.layer.conv2.Conv3d_conv,"[48, 48, 3, 3, 3]","[1, 48, 128, 128, 128]",62208.0,1.304596e+11
4_encoder0.layer.InstanceNorm3d_norm2,-,"[1, 48, 128, 128, 128]",NaN,NaN
...,...,...,...,...
174_decoder1.conv_block.InstanceNorm3d_norm2,-,"[1, 48, 128, 128, 128]",NaN,NaN
175_decoder1.conv_block.conv3.Conv3d_conv,"[96, 48, 1, 1, 1]","[1, 48, 128, 128, 128]",4608.0,9.663676e+09
176_decoder1.conv_block.InstanceNorm3d_norm3,-,"[1, 48, 128, 128, 128]",NaN,NaN
